# Limpieza del conjunto de datos Original

In [12]:
!pip install pandas openpyxl
import pandas as pd
import os
from IPython.display import FileLink 

class DataProcessor:
    def __init__(self, file_name):
        self.file_name = file_name
        
    def load_data(self):
        return pd.read_excel(self.file_name)
    
    def clean_column_names(self, df):    
        column_mapping = {
            'DESCRIPCION_CONDUCTA': 'CONDUCTA',
            'FECHA_HECHO': 'FECHA',
            'DIA_SEMANA': 'DIA',
            'CLASE_SITIO': 'SITIO',
            'ESTADO_CIVIL_PERSONA': 'ESTADO_CIVIL',
            'CARGO_PERSONA': 'CARGO',
            'GRADO_INSTRUCCION_PERSONA': 'ESCOLARIDAD',
            'ARMAS_MEDIOS': 'ARMAS'
        }
        return df.rename(columns=column_mapping)

    def drop_columns(self, df):
        df = df.drop(df.index[-1])
        columns_to_drop = [
            'MUNICIPIO_HECHO',
            'BARRIOS_HECHO',
            'JURIS.DISTRITO / SECCIONAL',
            'JURIS.CAI',
            'MES_LARGO',
            'AÑO',
            'SEMANA_HECHO',
            'HORA_HECHO',
            'INTERVALOS_HORA',
            'DIRECCION_HECHO',
            'LATITUD',
            'LONGITUD',
            'GENERO',
            'CLASE_EMPLEADO_DESCRIPCION',
            'DEL 01/01/2022 AL 15/11/2022'
        ]
        return df.drop(columns=columns_to_drop)

    def clean_dataset(self, df):
        df = self.clean_column_names(df)
        df = self.drop_columns(df)
        
        # Tratar los caracteres especiales de Modalidad
        mapeo_modalidad = {'-': 'OTRO' }
        df['MODALIDAD'] = df['MODALIDAD'].replace(mapeo_modalidad)

        # Limpiar la columna de edad
        df['EDAD'] = df['EDAD'].replace('-', pd.NaT)
        df['EDAD'] = pd.to_numeric(df['EDAD'], errors='coerce')
        promedio_edades = df['EDAD'].mean(skipna=True)
        df['EDAD'] = df['EDAD'].fillna(promedio_edades)

        # Eliminar caracteres innecesarios de los registros
        df['CONDUCTA'] = df['CONDUCTA'].astype(str)
        df['CONDUCTA'] = df['CONDUCTA'].apply(lambda x: x.split('.', 1)[-1].strip() if '.' in x else x)

        # Convertir la columna de fecha al formato datetime
        df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
        # Crear nuevas columnas para mes y año
        df['Mes'] = df['FECHA'].dt.month
        df['Año'] = df['FECHA'].dt.year

        # Eliminar la columna original de fecha si ya no es necesaria
        df = df.drop(columns=['FECHA'])
        return df

    def save_and_download_clean_data(self, df):
        try:
            # Obtener la ruta del directorio actual
            ruta_actual = os.getcwd()
            # Concatenar el nombre del archivo de salida al final de la ruta actual
            archivo_salida = os.path.join(ruta_actual, 'datasetClean.xlsx')
            # Guardar el DataFrame en el archivo Excel en la ruta especificada
            df.to_excel(archivo_salida, index=False)
            # Crear un enlace para descargar el archivo generado
            link = FileLink(archivo_salida)
            # Mostrar el enlace para descargar el archivo
            print("Enlace para descargar el archivo:")
            display(link)
        except Exception as e:
            print(f"Error al guardar el archivo: {e}")


    def process_data(self):
        # Cargar datos
        df = self.load_data()        
        # Limpiar datos
        df_cleaned = self.clean_dataset(df)        
        # Guardar y descargar datos limpios
        self.save_and_download_clean_data(df_cleaned)


if __name__ == "__main__":
    # Obtener la ruta del directorio actual
    ruta_actual = os.getcwd()
    # Concatenar el nombre del archivo al final de la ruta actual
    archivo = os.path.join(ruta_actual, 'violencias 2022.xlsx')
    # Instanciar la clase y procesar los datos
    processor = DataProcessor(archivo)
    processor.process_data()


Enlace para descargar el archivo:


C:\Users\klgt1\Desktop\Tesis\1.ETLData\datasetClean.xlsx

# Transformación de datos del conjunto de datos limpio.

In [13]:
!pip install holidays
import pandas as pd
import os
import holidays

class DataTransformer:
    def __init__(self, file_name):
        self.df = pd.read_excel(file_name)
    
    def transform(self):
        self._transform_conducta()
        self._transform_zona()
        self._transform_one_hot_movil_victima()
        self._transform_one_hot_movil_agresor()
        self._transform_one_hot_estado_civil()
        self._transform_one_hot_dia_semana()
        self._transform_one_hot_turno()
        self._transform_one_hot_armas()
        self._transform_frequency_sitio()
        self._transform_one_hot_cargo()
        self._transform_one_hot_modalidad()
        self._transform_label_encoding_escolaridad()
        self._add_special_events()
        self._add_holidays()
        self._save_data()
    
    def _transform_conducta(self):
        self.df['CONDUCTA'] = self.df['CONDUCTA'].str.strip()
        self.df['CONDUCTA'] = self.df['CONDUCTA'].map(lambda x: 1 if x in ['HOMICIDIO', 'FEMINICIDIO',"ACCESO CARNAL VIOLENTO",
                                                                           "ACTO SEXUAL VIOLENTO",
                                                                           "ACCESO CARNAL O ACTO SEXUAL EN PERSONA PUESTA EN INCAPACIDAD DE RESISTIR",
                                                                           "ACCESO CARNAL ABUSIVO CON MENOR DE 14 AÑOS",
                                                                           "ACTOS SEXUALES CON MENOR DE 14 AÑOS","ACOSO SEXUAL",
                                                                           "ACCESO CARNAL O ACTO SEXUAL ABUSIVO CON INCAPAZ DE RESISTIR"] else 0)
    
    def _transform_zona(self):        
        mapeo_zona = {'RURAL': 0, 'URBANA': 1}
        self.df['ZONA'] = self.df['ZONA'].map(mapeo_zona)

        
    def _transform_one_hot_movil_victima(self):
        self.df['MOVIL_VICTIMA'] = self.df['MOVIL_VICTIMA'].replace({'PASAJERO VEHICULO': 'PASAJERO',
                                                                 'CONDUCTOR MOTOCICLETA': 'CONDUCTOR',
                                                                 'CONDUCTOR VEHICULO': 'CONDUCTOR', 'A PIE': 'OTRO'})
        dummies = pd.get_dummies(self.df['MOVIL_VICTIMA'], prefix='MOVIL_VICTIMA', dtype=int)
        self.df = pd.concat([self.df, dummies], axis=1)
        self.df.drop(columns=['MOVIL_VICTIMA'], inplace=True)    
        return self.df

    def _transform_one_hot_movil_agresor(self):
        self.df['MOVIL_AGRESOR'] = self.df['MOVIL_AGRESOR'].replace({'PASAJERO MOTOCICLETA': 'PASAJERO',
                                                                     'CONDUCTOR MOTOCICLETA': 'CONDUCTOR', 'A PIE': 'OTRO'})
        dummies = pd.get_dummies(self.df['MOVIL_AGRESOR'], prefix='MOVIL_AGRESOR', dtype=int)
        self.df = pd.concat([self.df, dummies], axis=1)
        self.df.drop(columns=['MOVIL_AGRESOR'], inplace=True)    
        return self.df
        
    def _transform_one_hot_estado_civil(self):
        dummies = pd.get_dummies(self.df['ESTADO_CIVIL'], prefix='ESTADO_CIVIL', dtype=int)
        self.df = pd.concat([self.df, dummies], axis=1)
        self.df.drop(columns=['ESTADO_CIVIL'], inplace=True)    
        return self.df
    
    def _transform_one_hot_dia_semana(self):
        dummies = pd.get_dummies(self.df['DIA'], prefix='DIA', dtype=int)
        self.df = pd.concat([self.df, dummies], axis=1)
        self.df.drop(columns=['DIA'], inplace=True)    
        return self.df
    
    def _transform_one_hot_turno(self):
        self.df['TURNO'] = self.df['TURNO'].replace({'PRIMERO': 'MADRUGADA',
                                                     'SEGUNDO': 'MAÑANA',
                                                     'TERCERO': 'TARDE',
                                                     'CUARTO': 'NOCHE'})
        dummies = pd.get_dummies(self.df['TURNO'], prefix='TURNO', dtype=int)
        self.df = pd.concat([self.df, dummies], axis=1)
        self.df.drop(columns=['TURNO'], inplace=True)    
        return self.df


    def _transform_one_hot_armas(self):
        self.df['ARMAS'] = self.df['ARMAS'].replace({'ARMA DE FUEGO': 'ARMAS DE FUEGO/TRAUMATICAS', 'ARMA TRAUMATICA': 'ARMAS DE FUEGO/TRAUMATICAS',
                                                       'ESCOPOLAMINA': 'OTROS','ARMA BLANCA / CORTOPUNZANTE':'ARMA BLANCA/CONTUNDENTES',
                                                       'CONTUNDENTES':'ARMA BLANCA/CONTUNDENTES',
                                                       'ARTEFACTO EXPLOSIVO/CARGA DINAMITA': 'OTROS','CARRO BOMBA' : 'OTROS','PERRO':'OTROS'})
        dummies = pd.get_dummies(self.df['ARMAS'], prefix='ARMAS', dtype=int)
        self.df = pd.concat([self.df, dummies], axis=1)
        self.df.drop(columns=['ARMAS'], inplace=True)    
        return self.df
        
    def _transform_frequency_sitio(self):
        frequency_map = self.df['SITIO'].value_counts(normalize=True).to_dict()
        self.df['SITIO'] = self.df['SITIO'].map(frequency_map)
        return self.df
        
    def _transform_one_hot_cargo(self):
        self.df['CARGO'] = self.df['CARGO'].replace({'ESTUDIANTE': 'SIN INGRESOS', 'HABITANTE DE LA CALLE': 'SIN INGRESOS',
                                                  'AMA DE CASA': 'SIN INGRESOS', 'NO REPORTADO': 'OTROS',
                                                  'OFICIOS VARIOS': 'CON INGRESOS', 'VIGILANTE': 'CON INGRESOS',
                                                  'MESERO(A)': 'CON INGRESOS', 'AGRICULTOR': 'CON INGRESOS',
                                                  'TRABAJADOR SEXUAL': 'CON INGRESOS', 'TECNICO': 'CON INGRESOS',
                                                  'ABOGADOS  CLASE 1': 'CON INGRESOS', 'EMPLEADO': 'CON INGRESOS',
                                                  'COMERCIANTE': 'CON INGRESOS', 'DOCENTE': 'CON INGRESOS',
                                                  'PROMOTOR DE SALUD': 'CON INGRESOS',
                                                  'PATRULLERO POLICIA NACIONAL': 'CON INGRESOS',
                                                  'ESTILISTA': 'CON INGRESOS', 'INDEPENDIENTE': 'CON INGRESOS'})
        dummies = pd.get_dummies(self.df['CARGO'], prefix='CARGO', dtype=int)
        self.df = pd.concat([self.df, dummies], axis=1)
        self.df.drop(columns=['CARGO'], inplace=True)    
        return self.df

    def _transform_one_hot_modalidad(self):
        self.df['MODALIDAD'] = self.df['MODALIDAD'].replace({'SICARIATO': 'VIOLENCIA FÍSICA', 
                                                         'SIGNOS DE VIOLENCIA FÍSICA (TORTURA)': 'VIOLENCIA FÍSICA',
                                                         'ATRACO': 'VIOLENCIA FÍSICA', 
                                                         'RIÑAS': 'VIOLENCIA FÍSICA',
                                                         'FORCEJEO': 'VIOLENCIA FÍSICA',
                                                         'RIÑA O CONFLICTO ENTRE PANDILLAS': 'VIOLENCIA FÍSICA',
                                                         'EMPLEO DE SUSTANCIAS TOXICAS': 'VIOLENCIA FÍSICA',
                                                         'RIÑA ENTRE COMPAÑEROS PERMANENTES': 'VIOLENCIA INTRAFAMILIAR', 
                                                         'VIOLENCIA INTRAFAMILIAR': 'VIOLENCIA INTRAFAMILIAR',
                                                         'RIÑA ENTRE ESPOSOS': 'VIOLENCIA INTRAFAMILIAR', 
                                                         'RIÑA HIJO-MADRE': 'VIOLENCIA INTRAFAMILIAR',
                                                         'RIÑA ENTRE HIJO-PADRE': 'VIOLENCIA INTRAFAMILIAR',
                                                         'RIÑA ENTRE HERMANOS': 'VIOLENCIA INTRAFAMILIAR',
                                                         'VIOLENCIA SEXUAL': 'VIOLENCIA SEXUAL', 
                                                         'AMENAZA/CHANTAJE': 'VIOLENCIA PSICOLOGICA',
                                                         'POR AMIGO / CONOCIDO': 'VIOLENCIA PSICOLOGICA', 
                                                         'POR ESTABLECER': 'OTROS',
                                                         'NO REPORTADA': 'OTROS', 
                                                         'ACCIDENTAL': 'OTROS',
                                                         'CREACION DE SITIOS WEB': 'OTROS',
                                                         'APLICACIONES': 'OTROS', 
                                                         'TERRORISMO': 'OTROS',
                                                         'OTRO': 'OTROS'})
        dummies = pd.get_dummies(self.df['MODALIDAD'], prefix='MODALIDAD', dtype=int)
        self.df = pd.concat([self.df, dummies], axis=1)
        self.df.drop(columns=['MODALIDAD'], inplace=True)    
        return self.df

    def _transform_label_encoding_escolaridad(self):
        orden_escolaridad = ['ANALFABETA', 'PRIMARIA', 'SECUNDARIA', 'TECNICO', 'TECNOLOGO', 'SUPERIOR', 'NO REPORTADO']
        self.df['ESCOLARIDAD'] = self.df['ESCOLARIDAD'].astype(pd.CategoricalDtype(categories=orden_escolaridad, ordered=True)).cat.codes
        return self.df
    
    def _add_special_events(self):
        fechas_especiales = pd.to_datetime(['2022-01-06','2022-02-14', '2022-03-08', '2022-05-08', '2022-06-19', '2022-07-20', '2022-09-17','2022-10-31','2022-12-07','2022-12-24'])
        fechas_especiales_por_mes = [sum(1 for fecha in fechas_especiales if fecha.month == mes) for mes in range(1, 13)]
        self.df['Mes'] = self.df['Mes'].astype(int)
        self.df['eventos_especiales_mes'] = self.df['Mes'].apply(lambda mes: 1 if fechas_especiales_por_mes[mes - 1] != 0 else 0)

    
    def _add_holidays(self):
        festivos_colombia = holidays.Colombia(years=2022)
        fechas = pd.date_range(start='2022-01-01', end='2022-12-31', freq='D')
        festivos_por_mes = [sum(1 for fecha in fechas if fecha in festivos_colombia and fecha.month == mes) for mes in range(1, 13)]
        self.df['dias_festivos_mes'] = self.df['Mes'].apply(lambda mes: 1 if festivos_por_mes[mes - 1] != 0 else 0)
        self.df = self.df.drop('Mes', axis=1)
    
    def _save_data(self):
        try:
            '''self.df.to_excel('dataset_Transformado.xlsx', index=False)
            link = FileLink('dataset_Transformado.xlsx')
            print("Enlace para descargar el archivo:")
            display(link)'''
            # Obtener la ruta del directorio actual
            ruta_actual = os.getcwd()
            
            # Concatenar el nombre del archivo de salida al final de la ruta actual
            archivo_salida = os.path.join(ruta_actual, 'dataset_Transformado.xlsx')
            
            # Guardar el DataFrame en el archivo Excel en la ruta especificada
            self.df.to_excel(archivo_salida, index=False)
            
            # Crear un enlace para descargar el archivo generado
            link = FileLink(archivo_salida)
            
            # Mostrar el enlace para descargar el archivo
            print("Enlace para descargar el archivo:")
            display(link)
        except Exception as e:
            print(f"Error al guardar el archivo: {e}")

if __name__ == "__main__":
    # Obtener la ruta del directorio actual
    ruta_actual = os.getcwd()
    # Concatenar el nombre del archivo al final de la ruta actual
    archivo = os.path.join(ruta_actual, 'datasetClean.xlsx')
    #transformer = DataTransformer("C:\\Users\\klgt1\\Downloads\\datasetClean.xlsx")
    transformer = DataTransformer(archivo)
    transformer.transform()


Enlace para descargar el archivo:


C:\Users\klgt1\Desktop\Tesis\1.ETLData\dataset_Transformado.xlsx

# Generar conjunto de datos balanceado

In [14]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from IPython.display import display, FileLink
import os

class DataBalancer:
    def __init__(self, dataframe):
        self.df = dataframe
        
    def balance_data(self, minority_class_size=407):
        X = self.df.drop('CONDUCTA', axis=1)
        y = self.df['CONDUCTA']

        n_neighbors_value = min(minority_class_size - 1, 10)
        smote = SMOTE(sampling_strategy=0.80, k_neighbors=n_neighbors_value, random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X, y)

        return X_resampled, y_resampled

    def save_balanced_data(self, X_resampled, y_resampled, filename='Balanced_Data_Set.xlsx'):
        try:
            # Obtener la ruta del directorio actual
            ruta_actual = os.getcwd()
            # Concatenar el nombre del archivo de salida al final de la ruta actual
            archivo_salida = os.path.join(ruta_actual, filename)
            
            X_resampled['CONDUCTA'] = y_resampled
            X_resampled.to_excel(archivo_salida, index=False)
            link = FileLink(archivo_salida)
            print("Enlace para descargar el archivo:")
            display(link)
        except Exception as e:
            print(f"Error al guardar el archivo: {e}")

    def print_variable_count(self, X_resampled):
        print(f"Número de variables en el conjunto de datos balanceado: {X_resampled.shape[1]}")

    def process_data(self):
        # Balancear los datos
        X_resampled, y_resampled = self.balance_data()
        # Verifica el balance de las clases después del balanceo
        print("Clases equilibradas después de SMOTE:", pd.Series(y_resampled).value_counts())
        # Guarda el nuevo dataset
        self.save_balanced_data(X_resampled, y_resampled)
        # Imprime el número de variables
        self.print_variable_count(X_resampled)
        
if __name__ == "__main__":
    # Obtener la ruta del directorio actual
    ruta_actual = os.getcwd()
    # Concatenar el nombre del archivo al final de la ruta actual
    archivo = os.path.join(ruta_actual, 'dataset_Transformado.xlsx')
    # Cargar datos
    df = pd.read_excel(archivo)
    # Instanciar la clase y procesar los datos
    balancer = DataBalancer(df)
    balancer.process_data()


C:\Users\klgt1\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\klgt1\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


Clases equilibradas después de SMOTE: CONDUCTA
0    1404
1    1123
Name: count, dtype: int64
Enlace para descargar el archivo:


C:\Users\klgt1\Desktop\Tesis\1.ETLData\Balanced_Data_Set.xlsx

Número de variables en el conjunto de datos balanceado: 43
